# 🔧 Etapa 2 — Pré-processamento e Feature Engineering
**Objetivo:** limpar, padronizar e transformar o dataset para modelagem.  
**Entregáveis:** `notebooks/02_Preprocessamento.ipynb` e `data/processed/dataset_clean.csv`.


In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold

# Carrega o mesmo dataset da EDA
raw = pd.read_csv("../data/datasets/students_performance.csv")
df = raw.copy()
df.head()


,student_id,age,gender,parental_education,study_hours_week,attendance_rate,extracurricular,sleep_hours,previous_scores,tutoring,internet_quality,family_income,health_status,final_grade
0,STD01416,22,M,bachelor,5.66,69.55,Yes,6.49,49.41,No,Good,Medium,Good,85.39
1,STD01345,19,M,bachelor,13.30,58.82,Yes,8.12,50.29,No,NaN,Medium,Good,98.43
2,STD01705,25,M,master,10.43,59.72,Yes,6.60,71.64,No,Good,Medium,Good,100.00
3,STD00299,21,F,bachelor,3.90,68.33,Yes,6.99,50.93,Yes,Poor,Low,Excellent,86.85
4,STD01762,19,F,bachelor,4.24,50.44,Yes,8.06,54.00,No,Poor,Medium,Excellent,83.25


## Decisões de pré-processamento (baseadas na EDA)

**Problemas observados na EDA:**
- `study_hours_week` com valores **negativos** e **>100** (inconsistência);
- `attendance_rate` com valores **>100** (inconsistência);
- `sleep_hours` com valores extremos (≥ 15) pouco plausíveis;  
- `age` com outlier **99** (provável erro);
- *Missing* em: `study_hours_week`, `family_income`, `sleep_hours`, `attendance_rate`, `internet_quality`, `previous_scores`.

**Política adotada:**
- Valores **fora de faixa** → marcar como `NaN` para **imputar** depois;
- Numéricas → **mediana** (robusta a outliers);
- Categóricas → **moda** (valor mais frequente);
- Encoding → **One-Hot** para todas as nominais (evita ordens artificiais);
- Escalonamento → **StandardScaler** nas numéricas;
- Feature engineering → pelo menos 3 novas features úteis à nota final.


In [2]:
# 4.1 – Corrige faixas impossíveis antes da imputação (vira NaN e depois a gente imputa)
df.loc[(df['study_hours_week'] < 0) | (df['study_hours_week'] > 100), 'study_hours_week'] = np.nan
df.loc[(df['attendance_rate'] < 0) | (df['attendance_rate'] > 100), 'attendance_rate'] = np.nan
df.loc[(df['sleep_hours'] < 3) | (df['sleep_hours'] > 16), 'sleep_hours'] = np.nan
df.loc[(df['age'] < 15) | (df['age'] > 60), 'age'] = np.nan  # remove o 99 e outros absurdos

# 4.2 – Define colunas por tipo
target = 'final_grade'
id_col = 'student_id'

numeric_cols = ['age','study_hours_week','attendance_rate','sleep_hours','previous_scores']
categorical_cols = ['gender','parental_education','extracurricular','tutoring',
                    'internet_quality','family_income','health_status']

# Sanidade: garante que todas existem
missing_report = [c for c in numeric_cols+categorical_cols+[target,id_col] if c not in df.columns]
missing_report


[]

In [3]:
df_fe = df.copy()

# 5.1 Eficiência de estudo: nota anterior por horas de estudo (evita div/0 com "+1")
df_fe['study_efficiency'] = df_fe['previous_scores'] / (df_fe['study_hours_week'] + 1)

# 5.2 Déficit de sono: quanto falta para 8 horas (clampa mínimo em 0)
df_fe['sleep_deficit'] = np.clip(8 - df_fe['sleep_hours'], 0, None)

# 5.3 Presença alta (marcador binário)
df_fe['high_attendance'] = (df_fe['attendance_rate'] >= 75).astype(float)

# (Opcional 4ª) Interação: estudo x presença (tende a ser útil)
df_fe['study_x_attendance'] = df_fe['study_hours_week'] * df_fe['attendance_rate']

# Atualiza lista de numéricas com novas colunas
numeric_plus = numeric_cols + ['study_efficiency','sleep_deficit','high_attendance','study_x_attendance']


In [4]:
# Separa features e alvo (mantém o target fora do transformer)
X = df_fe.drop(columns=[target, id_col], errors='ignore')
y = df_fe[target].copy()

# Garante listas consistentes com X
num_used = [c for c in numeric_plus if c in X.columns]
cat_used = [c for c in categorical_cols if c in X.columns]

# Pipelines
num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

pre = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_used),
        ("cat", cat_pipe, cat_used)
    ],
    remainder='drop'
)

# Ajusta e transforma
X_t = pre.fit_transform(X)

# Reconstrói nomes de colunas após One-Hot
ohe = pre.named_transformers_['cat'].named_steps['onehot']
cat_names = ohe.get_feature_names_out(cat_used)
X_cols = np.concatenate([np.array(num_used), cat_names])

X_proc = pd.DataFrame(X_t, columns=X_cols, index=X.index)

# Remove colunas com variância zero (se houver)
vt = VarianceThreshold(threshold=0.0)
X_proc = pd.DataFrame(vt.fit_transform(X_proc), columns=X_cols[vt.get_support()], index=X.index)

X_proc.head()


,age,study_hours_week,attendance_rate,sleep_hours,previous_scores,study_efficiency,sleep_deficit,high_attendance,study_x_attendance,gender_ M,...,family_income_Medium,health_status_ Excellent,health_status_ Good,health_status_ Poor,health_status_EXCELLENT,health_status_Excellent,health_status_GOOD,health_status_Good,health_status_POOR,health_status_Poor
0,0.177274,-0.707279,1.044461,-0.429122,-0.850525,0.134587,0.400791,-0.221117,-0.535456,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.130141,0.454930,-0.069411,0.899028,-0.755216,-0.576413,-1.192811,-0.221117,0.455886,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.484688,0.018341,0.024018,-0.339493,1.557113,-0.075171,0.284701,-0.221117,0.049235,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.258531,-0.975013,0.917814,-0.021714,-0.685901,0.676648,-0.126892,-0.221117,-0.859820,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-1.130141,-0.923292,-0.939331,0.850139,-0.353402,0.660516,-1.192811,-0.221117,-0.994042,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Justificativas de tratamento

- **Imputação numérica (mediana):** robusta a outliers e adequada para assimetria.
- **Imputação categórica (moda):** preserva a categoria mais frequente quando há lacunas.
- **One-Hot Encoding:** evita ordens artificiais; adequado para variáveis nominais.
- **StandardScaler (numéricas):** centraliza e padroniza, útil para modelos sensíveis à escala.
- **VarianceThreshold:** remove colunas sem variação (não agregam informação).
- **Regras de faixa:** valores impossíveis viram `NaN` e são imputados (evita distorções).


In [5]:
# Split 70/15/15
X_train, X_temp, y_train, y_temp = train_test_split(
    X_proc, y, test_size=0.30, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)

# Monta um único DataFrame "limpo" (features + target) para entregar
df_clean = X_proc.copy()
df_clean[target] = y

# Garante pasta e salva
import os
os.makedirs("../data/processed", exist_ok=True)
df_clean.to_csv("../data/processed/dataset_clean.csv", index=False)

print("Dimensões (X_train, X_val, X_test):", X_train.shape, X_val.shape, X_test.shape)
print("Arquivo salvo em: data/processed/dataset_clean.csv")


Dimensões (X_train, X_val, X_test): (1757, 67) (376, 67) (377, 67)
Arquivo salvo em: data/processed/dataset_clean.csv


### Comparação: antes vs depois

- **Antes:** *missing values* em `study_hours_week`, `attendance_rate`, `sleep_hours`, `previous_scores`, `family_income`, `internet_quality` e outliers (idade 99, estudo negativo, frequência >100).
- **Depois:** dataset **sem valores faltantes**, variáveis categóricas codificadas via **One-Hot**, numéricas **padronizadas** e com **features novas**.
- **Pronto para modelagem:** o arquivo `data/processed/dataset_clean.csv` está pronto para ser usado na Etapa 3 (Modelagem) sem novos tratamentos.
